# 케라스에서 뉴럴 스타일 트랜스퍼 구현하기

In [1]:
import os
import tensorflow as tf

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# 변수 초깃값 정의하기
from keras.preprocessing.image import load_img, img_to_array, save_img

target_image_path = './datasets/portrait.png' # 변환하려는 이미지 경로
style_reference_image_path = './datasets/popova.jpg' # 스타일 이미지 경로

width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

In [3]:
# 유틸리티 함수
import numpy as np
from keras.applications import vgg19

def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68 # ImageNet의 평균 픽셀 값을 더함. vgg19.preprocess_input 함수에서 일어나는 변환을 복원.
    x = x[:, :, ::-1] # 이미지를 'BGR'에서 'RGB'로 변환. 이것도 vgg19.preprocess_input 함수에서 일어나는 변환을 복원하기 위함.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [4]:
# 사전 훈련된 VGG19 네트워크를 로딩하고 3개 이미지에 적용하기
from keras import backend as K

target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3)) # 생성된 이미지를 담을 플레이스홀더

input_tensor = K.concatenate([target_image, style_reference_image, combination_image], axis=0) # 3개의 이미지를 하나의 배치로 합침.

model = vgg19.VGG19(input_tensor=input_tensor,
                   weights='imagenet',
                   include_top=False) # 세 이미지의 배치를 입력으로 받는 VGG 네트워크를 만듦. 이 모델은 사전 훈련된 ImageNet 가중치를 로드.
print('모델 로드 완료.')

80142336/80134624 [==============================] - 7s 0us/step
모델 로드 완료.


In [5]:
# 콘텐츠 손실
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [6]:
# 스타일 손실
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2,0,1)))
    gram = K.dot(features, K.transpose(features))
    return gram

def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C) / (4. * (channels ** 2) * (size ** 2)))

In [7]:
# 총 변위 손실(variation loss)
def total_variation_loss(x):
    '''
    생성된 이미지의 픽셀을 사용하여 계산. 생성된 이미지가 공간적인 연속성을 가지도록 도와주며 픽셀의 격자무늬가 과도하게 나타나는 것을 막아줌.
    일종의 규제 항.
    '''
    a = K.square(
        x[:, :img_height - 1, :img_width - 1, :]-
        x[:, 1:, :img_width - 1, :])
    b = K.square(
        x[:, :img_height - 1, :img_width - 1, :]-
        x[:, :img_height - 1, 1: , :])
    return K.sum(K.pow(a+b, 1.25))

In [9]:
# 최소화할 최종 손실 정의하기
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers]) # 층 이름과 활성화 텐서를 매핑한 딕셔너리

content_layer = 'block5_conv2' # 콘텐츠 손실에 사용할 층
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1'] # 스타일 손실에 사용할 층
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.025 # 손실 항목의 가중치 평균에 사용할 가중치

loss = K.variable(0.) # 모든 손실 요소를 더하여 하나의 스칼라 변수로 손실을 정의.
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features, combination_features)

for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1,:,:,:]
    combination_features = layer_features[2,:,:,:]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight/len(style_layers)) * sl
    
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [10]:
# 경사 하강법 단계 설정하기
grads = K.gradients(loss, combination_image)[0] # 손실에 대한 생성된 이미지의 그래디언트를 구함.

fetch_loss_and_grads = K.function([combination_image], [loss, grads])

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
        
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values
    
evaluator = Evaluator()

In [11]:
# 스타일 트랜스퍼 반복 루프
from scipy.optimize import fmin_l_bfgs_b
import time

result_prefix = 'style_transfer_result'
iterations = 20

x = preprocess_image(target_image_path) # 초깃값은 타깃 이미지
x = x.flatten()
for i in range(iterations):
    print('반복 횟수:', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss,
                                    x,
                                    fprime=evaluator.grads,
                                    maxfun=20)
    print('현재 손실 값:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    save_img(fname, img)
    print('저장 이미지:', fname)
    end_time = time.time()
    print('%d 번째 반복 완료: %ds' % (i, end_time - start_time))

반복 횟수: 0
현재 손실 값: 8099096000.0
저장 이미지: style_transfer_result_at_iteration_0.png
0 번째 반복 완료: 7s
반복 횟수: 1
현재 손실 값: 3367371500.0
저장 이미지: style_transfer_result_at_iteration_1.png
1 번째 반복 완료: 3s
반복 횟수: 2
현재 손실 값: 2109817100.0
저장 이미지: style_transfer_result_at_iteration_2.png
2 번째 반복 완료: 3s
반복 횟수: 3
현재 손실 값: 1598989300.0
저장 이미지: style_transfer_result_at_iteration_3.png
3 번째 반복 완료: 3s
반복 횟수: 4
현재 손실 값: 1302961200.0
저장 이미지: style_transfer_result_at_iteration_4.png
4 번째 반복 완료: 3s
반복 횟수: 5
현재 손실 값: 1127634800.0
저장 이미지: style_transfer_result_at_iteration_5.png
5 번째 반복 완료: 3s
반복 횟수: 6
현재 손실 값: 1000282240.0
저장 이미지: style_transfer_result_at_iteration_6.png
6 번째 반복 완료: 4s
반복 횟수: 7
현재 손실 값: 911152500.0
저장 이미지: style_transfer_result_at_iteration_7.png
7 번째 반복 완료: 3s
반복 횟수: 8
현재 손실 값: 837610500.0
저장 이미지: style_transfer_result_at_iteration_8.png
8 번째 반복 완료: 3s
반복 횟수: 9
현재 손실 값: 784922300.0
저장 이미지: style_transfer_result_at_iteration_9.png
9 번째 반복 완료: 3s
반복 횟수: 10
현재 손실 값: 738927940.0
저장 이미지: style_transfer